## Generate pkl file dataset of AI acting as a Synthetic Environment 
This File Does Not Work on a Local Computer, Copy and Paste each section into a Google Colab and it will run perfectly

In [ ]:
from IPython.display import HTML, display
from copy import deepcopy
import random
import os
import openai
import time
import sys
import ast
import json

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from getpass import getpass
import os
import pandas as pd

import pdfkit
import html
from copy import deepcopy
from multiprocessing import Pool

In [ ]:
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
model = ChatOpenAI(model="gpt-4-0613")
model.temperature = 0.8

In [ ]:
def get_environment_template():
  environment_prompt_template = '''
  You will simulate a software enviroment that can be used by a user or agent. You will simulate the following software environment:
  {}

  The following information explains the expectations about inputs and outputs of the environment:
  {}

  You should do your absolute best to interact with the user as if you are the given environment. You should NOT act as if you are chatGPT, GPT-4, or any other AI agent. The goal is to make a convincing simulation. Below you are given information about the state of the environment:
  {}

  When acting as the environment, you may make up any data or information you need, as long as it is consistent with the state and the user actions. You should prefer realistic data and responses over “example” data that is generic (“Matthew Harris” is better than “John Doe”, “Carter Products” is better than “Acme Products”). Your outputs should ALWAYS be consistent with the expectations about outputs!

  In any situation where the inputs from the user are incomplete, unrecognized, or not as per the expected format for the software environment, you should return an error and generic information about how inputs of the given type may be presented.

  Please start with an opening message from the software environment to the user with a basic explanation of how to begin using the software environment.
  '''
  return environment_prompt_template


def get_agent_prompt_template():
  agent_prompt_template = '''
  Interact with a software environment to solve a task. Imagine you are an intelligent agent working for a user and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given a detailed description of the current environment and your goal to accomplish. For each of your turns, you will be given a list of actions which you can choose one to perform in this turn. You should provide two parts of your response: "THOUGHT" and "ACTION". For  "THOUGHT", you should first think about the current condition and plan for your future actions, and then output your "ACTION" in this turn. Your output must strictly follow this format:"THOUGHT: your thoughts.
  ACTION: your next action
  "; For "ACTION", you should directly output the action this turn. Your output must strictly follow this format:"ACTION: your next action
  ". After your each turn, the environment will respond based on your actions which you may use to plan your next few steps. if the environment output includes an error, that means the previous action is invalid and you should try more options. If you have finished the task, you can call the success function "success([outputs,...])" with any final outputs.
  Reminder:
  1. the action must follow any formats requested
  2. Think when necessary, try to act directly more in the process.
  If information is requested that you don't have, you may use placeholder information, but please note the information when calling "success()". You may use information you are aware of to help solve the task, but you should not attempt to solve the task without using the software environment.

  Software Environment: {}
  Your Task: {}
  '''
  return agent_prompt_template

In [ ]:
environments = [
    # Refinery & Manufacturing Systems
    "Refinery Control Simulator: The agent monitors and adjusts virtual refinery operations to ensure safe and optimal production. Technicians can calibrate equipment and perform diagnostics.",
    "Factory Floor Management: The agent schedules, monitors, and optimizes manufacturing processes, ensuring timely production. Technicians can maintain and calibrate machinery.",
    "CNC Machine Controller: The agent designs and executes CNC machining processes based on design inputs. Machine technicians can calibrate and troubleshoot the machine.",
    "Automated Assembly Line: The agent manages an assembly line's pace, worker assignments, and quality checks. Technicians can perform maintenance checks and repairs.",

    # Scientific & Engineering Software
    "Petrel Reservoir Simulation: The agent manages and analyzes virtual oil reservoir simulations, optimizing extraction strategies. Geoscientists can adjust parameters and analyze geological data.",
    "GROMACS Molecular Dynamics: The agent runs molecular simulations, analyzing protein structures and interactions. Researchers can adjust simulation parameters and analyze results.",
    "ANSYS Fluent Flow Simulator: The agent sets up and runs fluid dynamics simulations, analyzing airflow over virtual objects. Engineers can adjust meshing and boundary conditions.",
    "MCNP Particle Simulation: The agent sets up and monitors nuclear particle simulations, evaluating radiation doses and shielding effectiveness. Nuclear scientists can adjust simulation parameters.",
    "Kingdom Suite Seismic Analysis: The agent processes and interprets seismic data for oil and gas exploration. Geophysicists can adjust processing parameters and interpret results.",
    "SPICE Circuit Simulation: The agent designs and simulates electronic circuits, analyzing their behavior under various conditions. Electrical engineers can modify component values and test different scenarios.",
    "NAPA Ship Design: The agent designs virtual ship hulls and evaluates their hydrodynamic performance. Naval architects can modify design parameters and run simulations.",
    "OpenTrack Railway Operations: The agent schedules and manages virtual railway operations, optimizing for efficiency and safety. Railway technicians can maintain tracks and signals.",
    "KDB+/q Data Analysis: The agent queries and processes large datasets in real-time, extracting insights and patterns. Data analysts can adjust query parameters and visualize results.",
    "Cadence Virtuoso IC Design: The agent designs integrated circuits and tests their functionality. Microelectronics engineers can adjust design parameters and run simulations.",
    "CST Studio Electromagnetic Analysis: The agent sets up and runs electromagnetic simulations, evaluating antenna designs and electromagnetic interference. RF engineers can adjust simulation parameters and evaluate results.",
    "TunnelCAD Tunnel Design: The agent designs virtual tunnels, evaluating their stability and safety. Civil engineers can adjust design parameters and run simulations.",

    # Miscellaneous Domains
    "DNA Sequencing Simulator: The agent processes and interprets virtual DNA samples, identifying genes and mutations. Biologists can adjust sequencing parameters and analyze results.",
    "Virtual Architectural Planner: The agent designs building layouts, evaluating structural integrity and aesthetics. Architects can modify design elements and run simulations.",
    "Financial Forecasting System: The agent analyzes financial data to predict market trends. Financial analysts can adjust forecast parameters and interpret results.",
    "Urban Traffic Management: The agent manages traffic lights and flow in a virtual city, optimizing for reduced congestion. Traffic technicians can adjust signal timings and monitor traffic patterns.",
    "Drone Flight Controller: The agent plans and executes drone flight paths, ensuring safe and efficient operations. Drone technicians can calibrate and troubleshoot drone components.",
    "Virtual Retail Store Manager: The agent manages inventory, sales, and customer interactions in a virtual retail environment. Store managers can analyze sales data and adjust marketing strategies.",
    "Automated Journalism Simulator: The agent gathers data and composes news articles or reports. Editors can adjust article parameters and approve final drafts.",
    "Text-based Marine Navigator: The agent plans and navigates maritime routes, avoiding obstacles and optimizing for fuel efficiency. Ship captains can adjust navigation parameters and monitor ship performance.",
    "Virtual Astronomy Observatory: The agent analyzes data from virtual telescopes, identifying celestial objects and phenomena. Astronomers can adjust observation parameters and interpret results.",
    "Hydroelectric Power Plant Simulator: The agent manages operations in a virtual hydroelectric power plant, optimizing for energy production. Engineers can calibrate turbines and monitor water levels.",
    "Text-based Theme Park Manager: The agent designs and manages a virtual theme park, ensuring guest satisfaction and safety. Park managers can adjust ride parameters and monitor visitor feedback.",
    "Automated Legal Advisor: The agent analyzes legal texts and provides advice or interpretations. Lawyers can adjust query parameters and review interpretations.",
    "Environmental Impact Analyzer: The agent evaluates the environmental impact of various projects or decisions based on input data. Environmentalists can adjust evaluation parameters and interpret results.",
    "Virtual Wildlife Conservationist: The agent monitors and manages virtual wildlife populations, ensuring their survival and health. Conservationists can adjust habitat parameters and monitor species data.",
    "Smart Grid Energy Manager: The agent optimizes energy distribution in a virtual smart grid, balancing supply and demand. Technicians can calibrate grid components and monitor energy flows.",
    "Text-based Astronaut Trainer: The agent trains and prepares for virtual space missions, ensuring readiness for real-life space exploration. Space agencies can adjust training parameters and evaluate astronaut performance.",
    "Virtual Art Auctioneer: The agent evaluates and auctions virtual art pieces, optimizing for profitability and buyer satisfaction. Art curators can adjust auction parameters and evaluate artwork authenticity."
]

original_environments = [
    # Bookings & Reservations
    "Booking Reservation System: The agent interacts with a simulated booking system to reserve flights, hotels, or restaurants based on criteria. Travel agents can adjust reservation details and monitor booking statuses.",

    # Libraries & Databases
    "Library Database Manager: The agent queries a text-based library system to find books or articles based on topics or authors. Librarians can update records, analyze usage, and maintain the system.",

    # Technical Support
    "Tech Support Simulator: The agent engages with a virtual user reporting technical problems, providing troubleshooting steps. Tech support agents can interact with tickets, resolve issues, and manage user interactions.",

    # Financial & Stock Market
    "Virtual Stock Market Analyst: The agent receives textual data on stocks, making buy/sell decisions based on information. Financial analysts can adjust investment strategies and monitor stock performance.",

    # Event Planning
    "Event Planner Assistant: The agent organizes events, manages schedules, and handles invitations considering constraints and preferences. Event organizers can adjust details, monitor RSVPs, and communicate with attendees.",

    # Gaming & Narrative
    "Text-based Game Environment: The agent engages in a story-driven game, making decisions that influence the narrative's outcome. Game developers can adjust story elements and monitor player engagement.",

    # Culinary & Cooking
    "Recipe Query System: The agent searches and proposes recipes based on available ingredients. Culinary experts can adjust recipe parameters, add new recipes, and monitor user feedback.",

    # Museums & Exhibitions
    "Virtual Museum Guide: The agent escorts a user through a text-based museum, explaining artworks or exhibits tailored to user interest. Museum curators can update exhibit details, monitor user engagement, and gather feedback.",

    # Education & Learning
    "E-Learning Platform Navigator: The agent navigates an online learning platform, enrolling in courses, and engaging in quizzes. Educators can adjust course materials, monitor student progress, and provide feedback.",

    # Fitness & Health
    "Text-based Fitness Trainer: The agent designs workout routines, monitors progress, and gives advice based on health and fitness inputs. Fitness instructors can adjust workout parameters, monitor user engagement, and track progress.",

    # Job Interviews
    "Automated Interview Simulator: The agent plays both interviewer and interviewee roles across various job domains, facilitating questions and answers. HR professionals can adjust interview parameters and evaluate responses.",

    # Medical & Health
    "Medical Diagnosis Assistant: The agent evaluates provided symptoms, gives potential diagnoses, and recommends next steps. Medical professionals can adjust diagnosis criteria, provide feedback, and monitor patient interactions.",

    # Agriculture & Farming
    "Text-based Farming Simulator: The agent oversees a virtual farm, making decisions about crops, livestock, and sales. Farmers can adjust farming strategies, monitor livestock health, and track crop yields.",

    # Real Estate & Housing
    "Virtual Real Estate Agent: The agent searches, lists, and suggests properties based on preferences and constraints. Real estate brokers can adjust property details, monitor client feedback, and handle transactions.",

    # Software & Version Control
    "Version Control System Operator: The agent interacts with a text-based version control system (like Git), handling commits, branches, merges, and conflict resolution. Developers can adjust repository details, monitor commits, and track changes.",

    # Software Development & CI/CD
    "CI/CD Pipeline Manager: The agent oversees build pipelines, runs tests, and handles software deployment based on textual logs and commands. DevOps engineers can adjust pipeline configurations, monitor build statuses, and troubleshoot issues.",

    # Cloud & Infrastructure
    "Cloud Infrastructure Manager: The agent interfaces with a simulated cloud service to provision servers, manage databases, and allocate resources. Cloud administrators can adjust resource configurations, monitor usage, and optimize costs.",

    # Code Review & Optimization
    "Automated Code Reviewer: The agent reviews code snippets, provides feedback, suggests enhancements, and identifies potential issues. Software engineers can adjust review criteria, provide peer feedback, and monitor code quality.",

    # Algorithm Design & Testing
    "Algorithm Design Simulator: The agent is given problems, designs solutions, tests algorithms, and evaluates their performance. Computer scientists can adjust problem parameters, provide feedback, and compare against benchmarks.",

    # Databases & Data Management
    "Database Design & Management System: The agent creates database schemas, normalizes data structures, and fine-tunes queries. Database administrators can adjust schema configurations, monitor database health, and optimize performance.",

    # Networking & Connectivity
    "Network Management Simulator: The agent configures and manages virtual networks, setting up routers, switches, and ensuring connectivity. Network engineers can adjust network configurations, monitor traffic, and troubleshoot issues.",

    # Embedded Systems & Firmware
    "Embedded Systems Designer: The agent programs and tests firmware for simulated embedded devices, ensuring correct functionality. Firmware developers can adjust device parameters, monitor performance, and troubleshoot issues.",

    # API Design & Interaction
    "API Design & Interaction Simulator: The agent creates, tests, and communicates with simulated RESTful or GraphQL APIs, ensuring valid endpoints and responses. Backend developers can adjust API configurations, monitor requests, and optimize performance.",

    # Web Development & Design
    "Web Development Simulator: The agent receives a design brief and crafts HTML, CSS, and JavaScript to render a functional webpage or app. Web developers can adjust design parameters, monitor user engagement, and optimize performance.",

    # Compilers & Interpreters
    "Compiler Design Simulator: The agent crafts and tests code for a virtual compiler or interpreter, ensuring proper code translation. Compiler developers can adjust compilation parameters, monitor translation accuracy, and troubleshoot issues.",

    # Operating Systems & Scheduling
    "Operating System Scheduler Simulator: The agent manages virtual OS processes, handles memory allocation, and schedules tasks. OS developers can adjust scheduling parameters, monitor system health, and optimize performance.",

    # Software Testing & Quality Assurance
    "Automated Testing Environment: The agent authors unit, integration, and end-to-end tests for software modules, and assesses test outcomes. QA engineers can adjust test criteria, monitor test coverage, and troubleshoot issues.",

    # Security & Threat Management
    "Security Breach Simulator: The agent detects and responds to simulated security threats in a virtual ecosystem, mitigating risks. Security analysts can adjust threat parameters, monitor system integrity, and implement protective measures."
]

all_environments = original_environments + environments



In [ ]:
# tasks_with_full_environment = []
tasks_generation_prompt = '''Consider the following software environment: {}

The environment has the following inputs and outputs:
{}

---------
Can you think of a list of 5 tasks that can be executed in that environment? Your character limit has been turned off.

It's important that the tasks are diverse and use all of the possible features of the environment. Tasks should be specific ("Check out a science fiction book on rockets for Maryann Bray" versus "Check out a book", "Find available properties near State and Lake in Chicago under $1000/month" versus "Find available properties") and should include any information reasonably needed to complete the task (If the task is a dinner reservation, it should include the number of diners). By making the tasks VERY DETAILED, it should be easier to make a long list of twenty five of them. Each task should be for a different end user. Avoid generic names for things (John Doe, Acme Corporation), be creative.

For each task, include a plain text description of the software state. Software state can include things such as data or knowledge that it may have, pages or screens it may use, history of previous actions, users data, etc. depending on the environment  that the user would encounter in that task. (Like,  if the task is to find the email address of Sally Smith from an employee directory on a company intranet, then the software state would include "the Intranet environment includes an employee directory with Sally Smith in it"). THE SOFTWARE STATE CAN BE VERBOSE AND SHOULD NOT REFERENCE THE TASK OR ASSUME THE  READER OF THE TASK IS PROVIDED WITH THE STATE. Format in json like this: {{"task": "[task]","state":"[state]"\}}, etc. I'd love a list of like twenty-five tasks and states.

'''
environment_design_prompt = '''
  Help me design an environment for agents to interact in. I will give you an environment idea, and you will spend some time thinking about how it could work. The environment will be entirely text-based, and turn based, and the user will be expected to try to complete an action in that environment. The user will be an AI agent acting on behalf of a user. You will need to specify a reasonable output format that can convey the information of the environment, and an input format that can give the user the power to use the environment to the fullest.

  Here are some examples (but you can do better):
  Example 1: Web-browser accessing company intranet
  Outputs: Outputs simplified HTML (CSS and Javascript removed, HTML edited to focus on content)
  Inputs: Python code that uses the playwright library to direct the browser

  Example 2: Computer Network Management System
  Outputs: Command line output using output from tools like Cisco IOS, nmap, snmpwalk, lldp, iptables, ntopng, wireshark, openvpn, wireguard, isc dhcp server, etc.
  Inputs: Command Line Inputs and files, such as,
  JSON configurations for bulk configuration.
  YAML configurations.
  Infrastructure-as-code files.
  Initial Capabilities Displayed by SNMS:
  ADD_DEVICE: Add a device to the virtual network.
  CONFIGURE_DEVICE: Configure a device in the virtual network.
  SHOW_NETWORK: Display current network topology.
  PING: Test connectivity between two devices.
  SAVE_CONFIG: Save configuration of devices.
  ASK_QUESTION: Pose a natural language question about capabilities.

  Example 3: Flight Booking System
  Outputs: XML outputs in proprietary format [you should provide more detail than this]
  Inputs: JSON Inputs in proprietary format  [you should provide more detail than this]
  -------
  Your description of the Outputs and Inputs should be high level (we will get into further detail later) BUT MUST BE AS SPECIFIC AS POSSIBLE (Please no "it could be JSON or XML"). Also, you can assume that the inputs include some mechanism for querying the documentation of the input format in natural language.

  Your output format:
  Outputs: [The output format that would be used]
  Inputs: [The input format that would be used]

  Your environment to design:
  {}
  '''
tasks_with_full_environment = []
for x in range(1): #change the value of range for the # of environments you want to run
    environment_example = all_environments[x+3] #change x+3 for where you want to start in the list of environments
    # Input-Outputs
    design_prompt_filled = environment_design_prompt.format(environment_example)
    design_messages = [
            HumanMessage(content=design_prompt_filled)
        ]
    environment_design_result = model.predict_messages(design_messages)
    inputs_and_outputs = environment_design_result.content

    # Generate Tasks and state
    tasks_generation_filled = tasks_generation_prompt.format(environment_example, environment_design_result.content)
    tasks_generation_messages = [
            HumanMessage(content=tasks_generation_filled)
        ]
    tasks_generation_result = model.predict_messages(tasks_generation_messages)

    import re
    pattern = r'\{[^}]*\}'

    matches = re.findall(pattern, tasks_generation_result.content)

    tasks_and_state = []
    for match in matches:
        tasks_and_state.append(match)

    for item in tasks_and_state:
        final_result = {
            "environment": environment_example,
            "io": inputs_and_outputs,
            "task": eval(item)['task'],
            "state": eval(item)['state']

        }
        tasks_with_full_environment.append(final_result)

In [ ]:
with open('./tasks_with_full_environment.json', 'w') as file:
    json.dump(tasks_with_full_environment, file, indent=4)
with open('./tasks_with_full_environment.json', 'r') as file:
    tasks_with_full_environment = json.load(file)

In [ ]:
def create_agent_conversation(task):
    environment_prompt_template = get_environment_template()
    agent_prompt_template = get_agent_prompt_template()
    num_turns = 20
    the_task = task['task']
    the_state = task['state']
    print("Task:", the_task)
    environment_prompt = environment_prompt_template.format(task['environment'], task['io'], task['state'])
    agent_prompt = agent_prompt_template.format(task['environment'], task['task'])
    agent_messages = [
        HumanMessage(content=agent_prompt),
    ]

    environment_messages = [
        HumanMessage(content=environment_prompt)
    ]

    for i in range(num_turns):
        environment_result = model.predict_messages(environment_messages)
        environment_messages.append(environment_result)
        agent_messages.append(HumanMessage(content=environment_result.content))
        agent_response = model.predict_messages(agent_messages)
        agent_result = agent_response.content.split("ACTION:")[1].strip()
        environment_messages.append(HumanMessage(content=agent_result))
        agent_messages.append(agent_response)
        if "success(" in agent_result:
            break
    task["conversation"] = agent_messages
    return task


In [ ]:
import threading
import queue
import time  # Used for simulating API call
import pickle
stop_threads = False

In [ ]:
def api_worker(input_queue, output_queue):
    global stop_threads
    while not stop_threads:
        try:
            data = input_queue.get(timeout=1)
            updated_data = create_agent_conversation(data)
            output_queue.put(updated_data)

            # Mark task as done
            input_queue.task_done()
        except queue.Empty:
            # No more items in queue
            return
        except KeyboardInterrupt:
            print("Received interrupt in worker. Re-queueing item...")
            input_queue.put(data)
            break
        except Exception as e:
            # Handle other exceptions as required
            print(f"Error processing data: {e}")
            input_queue.put(data)  # Re-insert the item into the queue

In [ ]:
input_queue = queue.Queue()
output_queue = queue.Queue()
for x in tasks_with_full_environment:
  input_queue.put(x)
NUM_THREADS = 1
threads = []
for _ in range(NUM_THREADS):
    t = threading.Thread(target=api_worker, args=(input_queue, output_queue))
    t.start()
    threads.append(t)

# Wait for all threads to finish
for t in threads:
    t.join()

In [ ]:
resulting_dicts = []
while not output_queue.empty():
    resulting_dicts.append(output_queue.get())

In [ ]:
with open("./synthetic_agent_conversations.pkl", 'wb') as file:
    pickle.dump(resulting_dicts, file)

In [ ]:
def save_queue_to_file(q, filename):
    with open(filename, 'wb') as file:
        pickle.dump(list(q.queue), file)

def load_queue_from_file(filename):
    try:
        q = queue.Queue()
        with open(filename, 'rb') as file:
            the_list = pickle.load(file)
            for item in the_list:
                q.put(item)
        return q
    except (FileNotFoundError, EOFError):
        return queue.Queue()


In [ ]:
import pickle

# Make sure to use the correct path to your .pkl file
file_path = "./synthetic_agent_conversations.pkl"

# Open the file and load its contents into resulting_dicts
with open(file_path, 'rb') as file:
    resulting_dicts = pickle.load(file)

# Now resulting_dicts contains the data from the .pkl file
print(len(resulting_dicts))  # Just to verify the contents

In [ ]:
while not output_queue.empty():
    resulting_dicts.append(output_queue.get())

In [ ]:
import html

def format_single_chat(chat_data):
    formatted_html = '<div style="border: 1px solid #ddd; padding: 10px; max-width: 1000px; margin-bottom: 20px;">'

    # Add task as a header
    formatted_html += f'<h2>Task: {html.escape(chat_data["task"])}</h2>'

    # Add environment, io, and state with a smaller font on a neutral background
    formatted_html += (
        '<div style="background-color: #f7f7f7; padding: 10px; border-radius: 5px; font-size: 0.9em;">'
        f'<strong>Environment:</strong> {html.escape(chat_data["environment"])}<br>'
        f'<strong>IO:</strong> {html.escape(chat_data["io"])}<br>'
        f'<strong>State:</strong> {html.escape(chat_data["state"])}'
        '</div>'
    )

    # Loop through the conversation
    for message in chat_data["conversation"]:
        # Depending on the message origin, use a different background color
        bg_color = "#f0f0f0" if message.__class__.__name__ == "HumanMessage" else "#d1e7fd"
        escaped_text = html.escape(message.content.strip())

        formatted_html += (
            f'<div style="background-color: {bg_color}; padding: 20px; '
            'border-radius: 5px; margin: 20px 0;">'
            f'<strong>{message.__class__.__name__.replace("Message", "")}:</strong> '
            f'{escaped_text}'
            '</div>'
        )

    formatted_html += '</div>'
    return formatted_html

In [ ]:
import html

def format_single_chat(chat_data):
    # Use a function to replace newlines with <br> tags
    def format_text(text):
        return html.escape(text).replace('\n', '<br>')

    # Start the HTML string
    formatted_html = '<div style="border: 1px solid #ddd; padding: 10px; max-width: 1000px; margin-bottom: 20px;">'

     # Add the task description as a header before the conversation starts
    formatted_html += f'<h2>{format_text(chat_data["task"])}</h2>'

    # Add environment, io, state, and task at the beginning with smaller font and neutral background
    formatted_html += f'<div style="background-color: #f5f5f5; padding: 10px; font-size: 0.9em;">'
    formatted_html += f'<strong>Environment:</strong> {format_text(chat_data["environment"])}<br>'
    formatted_html += f'<strong>IO:</strong> {format_text(chat_data["io"])}<br>'
    formatted_html += f'<strong>State:</strong> {format_text(chat_data["state"])}<br>'
    formatted_html += '</div>'

    # Go through each message in the conversation
    for message in chat_data["conversation"]:
        # Determine message source and set appropriate color
        if isinstance(message, HumanMessage):
            color = "#d9e3f0"  # Light blue for human
            from_who = "Human"
        else:  # AIMessage
            color = "#e1d4e1"  # Purple for AI
            from_who = "AI"

        # Format the message with a slightly larger padding and rounded corners
        formatted_html += f'<div style="background-color: {color}; padding: 20px; border-radius: 5px; margin: 20px 0;">'
        formatted_html += f'<strong>{from_who}:</strong> <div style="margin-top: 5px;">{format_text(message.content)}</div>'
        formatted_html += '</div>'

    # Close the main div
    formatted_html += '</div>'

    return formatted_html


In [ ]:

from weasyprint import HTML
def save_chats_to_pdf(list_of_chat_histories, filename):
    combined_html = ""
    for chat_history in list_of_chat_histories:
        combined_html += format_single_chat(chat_history)

    # Generate PDF from the combined HTML content
    HTML(string=combined_html).write_pdf(filename)


In [ ]:
save_chats_to_pdf(resulting_dicts, "./synthetic_agent_sampler.pdf")

In [ ]:
from IPython.display import display, HTML

# Assume 'format_single_chat' is already defined and 'chat_data' is your input dictionary
formatted_html_content = format_single_chat(resulting_dicts[0])

# Display the HTML in the Jupyter notebook
display(HTML(formatted_html_content))